# Regresja

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import cross_val_predict
from sklearn import datasets, linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn import model_selection
from sklearn import  metrics
from statsmodels.formula.api import ols

Rozważmy zbiór Boston

In [5]:
boston = datasets.load_boston()
# print description
# print(boston.DESCR)

/home/zyndram/anaconda3/envs/podstawy_sztucznej_inteligencji/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative dat

In [6]:
print(boston.keys())
print(boston.data.shape)
print(boston.feature_names)

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])
(506, 13)
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [7]:
# get the data
boston_X = boston.data
boston_Y = boston.target

# Split the data into training/testing sets
boston_X_train = boston_X[:-50]
boston_X_test = boston_X[-50:]
 
# Split the targets into training/testing sets
boston_y_train = boston_Y[:-50]
boston_y_test = boston_Y[-50:]

In [8]:
bos=pd.DataFrame(boston.data[:-50])
bos.columns = boston.feature_names
bos['PRICE']=boston.target[:-50]
bos.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [9]:
bos1=pd.DataFrame(boston_X[-50:])
bos1.columns = boston.feature_names
bos1['PRICE']=boston.target[-50:]
bos1.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,4.66883,0.0,18.1,0.0,0.713,5.976,87.9,2.5806,24.0,666.0,20.2,10.48,19.01,12.7
1,8.20058,0.0,18.1,0.0,0.713,5.936,80.3,2.7792,24.0,666.0,20.2,3.50,16.94,13.5
2,7.75223,0.0,18.1,0.0,0.713,6.301,83.7,2.7831,24.0,666.0,20.2,272.21,16.23,14.9
3,6.80117,0.0,18.1,0.0,0.713,6.081,84.4,2.7175,24.0,666.0,20.2,396.90,14.70,20.0
4,4.81213,0.0,18.1,0.0,0.713,6.701,90.0,2.5975,24.0,666.0,20.2,255.23,16.42,16.4


In [10]:
corr = bos1.corr()

corr

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
CRIM,1.000000,NaN,0.201003,NaN,0.211382,0.045282,0.161483,0.118856,0.733008,0.529592,0.180582,-0.147766,0.185681,-0.057827
ZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
INDUS,0.201003,NaN,1.000000,NaN,0.207219,-0.147740,0.421844,-0.170231,0.256055,0.812731,0.295379,-0.146834,0.408161,-0.414676
CHAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOX,0.211382,NaN,0.207219,NaN,1.000000,0.026864,0.275541,-0.205316,0.279781,0.304850,0.034593,-0.503015,0.250875,-0.308019
RM,0.045282,NaN,-0.147740,NaN,0.026864,1.000000,0.090267,0.212857,0.218370,-0.077908,0.439612,0.098176,-0.651521,0.562242
AGE,0.161483,NaN,0.421844,NaN,0.275541,0.090267,1.000000,-0.590915,0.004245,0.236345,0.338677,-0.198093,0.376219,-0.570963
DIS,0.118856,NaN,-0.170231,NaN,-0.205316,0.212857,-0.590915,1.000000,0.516316,0.173004,0.001232,0.030635,-0.417711,0.458780
RAD,0.733008,NaN,0.256055,NaN,0.279781,0.218370,0.004245,0.516316,1.000000,0.734176,0.132078,-0.258563,0.085061,0.105992
TAX,0.529592,NaN,0.812731,NaN,0.304850,-0.077908,0.236345,0.173004,0.734176,1.000000,0.073417,-0.238775,0.411040,-0.232839


In [71]:
model = ols("PRICE ~ I(np.power(LSTAT, 0.01)) * I(np.power(AGE, 0.7)) * I(np.power(B, 0.5)) * I(NOX) * I(INDUS) * I(RM)", bos).fit()
# Print the summary
print((model.summary2()))

                                                                Results: Ordinary least squares
Model:                                             OLS                                             Adj. R-squared:                                    0.853    
Dependent Variable:                                PRICE                                           AIC:                                               2531.2012
Date:                                              2022-03-25 15:20                                BIC:                                               2795.0407
No. Observations:                                  456                                             Log-Likelihood:                                    -1201.6  
Df Model:                                          63                                              F-statistic:                                       42.92    
Df Residuals:                                      392                                             Prob 

In [72]:
from sklearn import  metrics
metrics.r2_score(bos.PRICE, model.predict(bos))

0.8733890215211415

In [74]:
from sklearn import  metrics
metrics.r2_score(bos1.PRICE, model.predict(bos1))

-1.3032234880423261